In [4]:
def maximimsation_marge(montant,status):
    taux_marge = 0.05
    if status == "TP":
        res = 0 # le fraudeur est détecté
    elif status == "TN":
        res = taux_marge * montant # un client honnête est accepté
    elif status == "FP":
        res = 0.7*taux_marge * montant # un client honnête est bloqué
    elif status == "FN": # un fraudeur est accepté
        if montant <= 20:
            res = 0
        elif montant <= 50:
            res = -0.2 * montant
        elif montant <= 100:
            res = -0.3 * montant
        elif montant <= 200:
            res = -0.5 * montant
        else:
            res = -0.8 * montant
    return res

In [5]:
def application_regles(y_test,y_pred):
    df = pd.DataFrame({'y_test': y_test, 'y_pred': y_pred, 'status': "null"})
    for index, row in df.iterrows():
        if row['y_test'] == 1 and row['y_pred'] == 1:
            df.at[index, 'status'] = "TP"
        elif row['y_test'] == 0 and row['y_pred'] == 0:
            df.at[index, 'status'] = "TN"
        elif row['y_test'] == 0 and row['y_pred'] == 1:
            df.at[index, 'status'] = "FP"
        elif row['y_test'] == 1 and row['y_pred'] == 0:
            df.at[index, 'status'] = "FN"
    return df

In [6]:
import pandas as pd

df_train = pd.read_csv("data/smote/dataframe_train.csv")
df_test = pd.read_csv("data/smote/dataframe_test.csv")

In [7]:
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import f1_score 
import pickle


# TEST !
#df_train = df_train[:1000]
#df_test = df_test[:500]
####################

# ne pas utiliser  code decision
X_train = df_train.drop(['FlagImpaye','CodeDecision'], axis=1)  
y_train = df_train['FlagImpaye']

X_test = df_test.drop(['FlagImpaye','CodeDecision'], axis=1)
y_test = df_test['FlagImpaye']

# Modèles
models = {
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machine': SVC(),
    'Neural Network': MLPClassifier(random_state=42),
    'Linear Discriminant Analysis': LinearDiscriminantAnalysis()
}

# Initialiser un DataFrame pour stocker les résultats
f1_df = pd.DataFrame(index=['SMOTE'], columns=models.keys())
marge_df = pd.DataFrame(index=['SMOTE'], columns=models.keys())

# Évaluer chaque modèle
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_pred, average='weighted')
    f1_df.at['SMOTE', model_name] = f1

        
    df_res = application_regles(y_test,y_pred)
    for row in df_res.itertuples():
        df_res.at[row.Index, 'marge'] = maximimsation_marge(X_test.at[row.Index, 'Montant'],row.status)
    total_marge = df_res['marge'].sum()

    marge_df.at['SMOTE', model_name] = total_marge

    print("\ F1 score du modèle sur l'ensemble de test :", f1)
    print(confusion_matrix(y_test, y_pred))

    # save models
    filename = 'models/' + model_name + '.pkl'
    with open(filename, 'wb') as file:
        pickle.dump(model, file)
    
    

\ F1 score du modèle sur l'ensemble de test : 0.6357689472572705
[[351346 389492]
 [  3586   2987]]


In [ ]:
marge_df.head()

In [ ]:
f1_df.head()

In [ ]:
df_train["FlagImpaye"].value_counts()

In [ ]:
df_test["FlagImpaye"].value_counts()

In [ ]:
# print(df_res)
# total_marge = df_res['marge'].sum()
# print(total_marge)

In [ ]:
# indice = []
# for row in df_res.itertuples():
#     # fraudeur non détecté
#     indice.append(row.y_test == 1 and row.y_pred == 0)

# df_filtered = df_res.loc[indice]
# print(df_filtered)
# print(sum(df_filtered['marge']))